In [48]:
import os
os.environ['USE_PYGEOS'] = '0'

import numpy as np
import pandas as pd 
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt

from temporal_curve import temporal_curve_24hr
from temporal_curve import future_ddfs

In [49]:
ARI = 100
ens_ptl  = 95  # 50% ensemble percentile

In [50]:
# Get regional polygons
gdf_Pilbara = gpd.read_file('/home/lusun/LuSUN/Data/riotinto_mash/catchment_boundary/watershed_WGS84_without_offshore_islands_filledholes_dissolved.shp')
gdf_Pilbara

,id,geometry
0,1,"POLYGON ((118.74750 -20.29250, 118.74750 -20.3..."


In [51]:
# Read historical regional DDF 
his_ddfs_file = r'/home/lusun/LuSUN/output/baseline_regional_ddfs_ARI' + str(ARI) + '_byPilbaraClimate.csv'
his_ddfs = pd.read_csv(his_ddfs_file, sep=',', index_col=0)
his_ddfs



,Pilbara
mins,
1.2,5.509012
1.8,8.790863
3.0,12.525744
4.2,16.124094
4.8,19.470707
10.2,32.566658
15.0,41.523441
19.8,48.164856
25.2,53.415867


In [52]:
# data_file = r"E:\Tuflow\Cal_rainfall_timeseries\data\New Zealand_Extreme_Precipitation_baseline_NIWA.nc"
# da_ddfs_100a = xr.open_dataset(data_file)['Extreme_Precipitation'].sel(
#     ARI=ARI).drop(["ARI"]).rename({"lat": "latitude", "lon":"longitude"})
# da_ddfs_100a.rio.write_crs("epsg:4326", inplace=True)
# dataset = xr.open_dataset(data_file)
# variable_names = list(dataset.variables)
# variable_names
# for var_name in ['ARI', 'Hrs']:
#     var_values = dataset[var_name].values
#     print(f"Variable: {var_name}")
#     print(f"Values: {var_values}")
#     print()

In [53]:
# read AR6 pattern
# ens_ptl  = 50  # 50% ensemble percentile
# gev_file = "data/New Zealand_Extreme_Precipitation_Pattern_AR6.nc"
gev_file = "data/AUS_Pilbara_Extreme_Precipitation_Pattern_AR6.nc"

In [54]:
dataset = xr.open_dataset(gev_file)
variable_names = list(dataset.variables)
variable_names

['ari', 'hrs', 'pth', 'CZ', 'pattern']

In [55]:
for var_name in variable_names[:-1]:
    var_values = dataset[var_name].values
    print(f"Variable: {var_name}")
    print(f"Values: {var_values}")
    print()

Variable: ari
Values: [  2   3   5  10  15  25  50 100 150 200 300 500]

Variable: hrs
Values: [  3   6  12  24  48  72  96 120 144 240]

Variable: pth
Values: [ 1.   5.  15.9 25.  50.  75.  84.1 95.  99. ]

Variable: CZ
Values: ['Pilbara']



In [56]:
# # Read CZ='NNI' region
# da_cf = xr.open_dataset(gev_file).pattern.sel(ari=ARI, 
#                                               pth=ens_ptl, 
#                                               method='nearest'
#                                              ).drop(['ari', 'pth']).sel(CZ="NNI").drop(['CZ'])
# # to pandas.DataFrame
# df_cf = da_cf.to_dataframe().rename({"pattern":"cf"}, axis=1)
# df_cf.index = np.round(df_cf.index*60, 4) # transform duration units to minutes
# df_cf.index.name = "mins"


# Read all CZ regions
da_cf = xr.open_dataset(gev_file).pattern.sel(ari=ARI, 
                                              pth=ens_ptl, 
                                              method='nearest'
                                             ).drop(['ari', 'pth'])  
# to pandas.DataFrame
df_cf = da_cf.to_dataframe().rename({"pattern":"cf"}, axis=1)
df_cf = df_cf.rename(index=lambda x: x * 60, level='hrs')
df_cf.index.names = ['CZ', 'min']


df_cf

cf
CZ      min             
Pilbara 180    28.187120
        360    28.567095
        720    28.947208
        1440   29.327461
        2880   29.707857
        4320   29.930439
        5760   30.088392
        7200   30.210926
        8640   30.311056
        14400  30.591648

In [57]:
df_cf.loc['Pilbara']

,cf
min,
180,28.187120
360,28.567095
720,28.947208
1440,29.327461
2880,29.707857
4320,29.930439
5760,30.088392
7200,30.210926
8640,30.311056


In [58]:
# Read AR6 GMT, mid
ptn_file = "data/IPCC_AR6_GMT_curves.csv"

df_dgmt = pd.read_csv(ptn_file, index_col=0)
# df_dgmt = df_dgmt.iloc[5::5, 0:len(df_dgmt.columns):3]
df_dgmt = df_dgmt.iloc[5:, 0:len(df_dgmt.columns):3]
df_dgmt.columns = [col_name.split(":")[0] for col_name in df_dgmt.columns]

df_dgmt

,SSP1-1.9,SSP1-2.6,SSP2-4.5,SSP3-7.0,SSP5-8.5
Year,,,,,
2010,0.21,0.17,0.12,0.12,0.13
2011,0.25,0.21,0.15,0.15,0.15
2012,0.28,0.24,0.18,0.17,0.18
2013,0.32,0.27,0.20,0.20,0.21
2014,0.35,0.30,0.23,0.23,0.24
...,...,...,...,...,...
2096,0.70,1.21,2.02,3.38,4.38
2097,0.70,1.21,2.02,3.43,4.45
2098,0.70,1.22,2.02,3.48,4.51


In [59]:
df_cf.loc['Pilbara']

,cf
min,
180,28.187120
360,28.567095
720,28.947208
1440,29.327461
2880,29.707857
4320,29.930439
5760,30.088392
7200,30.210926
8640,30.311056


In [60]:
# df_cf_oneindex = df_cf.reset_index(level='CZ')

In [61]:
df_dgmt.columns

Index(['SSP1-1.9', 'SSP1-2.6', 'SSP2-4.5', 'SSP3-7.0', 'SSP5-8.5'], dtype='object')

In [62]:
# Iterate by SSP senarios 
fut_ddfs_full = pd.DataFrame()
fut_cdfs_full = pd.DataFrame()
fut_pdfs_full = pd.DataFrame()
    
for column_name in df_dgmt.columns:
    
    column_data = df_dgmt[column_name]
    
    # for region in ['ESI', 'WSI', 'NSI', 'WNI', 'ENI', 'NNI']:
    for region in ['Pilbara']:
    
        fut_ddfs = pd.DataFrame()
        fut_cdfs = pd.DataFrame()
        fut_pdfs = pd.DataFrame()

        for index, row in column_data.iteritems():
            dgmt = row

            df_cf_region = df_cf.loc[region]

            his_ddfs_region = pd.DataFrame(his_ddfs[region])
            his_ddfs_region.rename(columns={region: 'ddf'}, inplace=True) 

            fut_ddfs_region = future_ddfs(dgmt, df_cf_region, his_ddfs_region)
            
            # Convert to PDF
            df_crrs_fut = temporal_curve_24hr(fut_ddfs_region)
            crrs_cdf_series_fut = pd.DataFrame(df_crrs_fut['crr'])
            crrs_pdf_series_fut = pd.DataFrame(df_crrs_fut['crr'].diff().fillna(df_crrs_fut['crr'][0]))
            
            fut_ddfs_region.rename(columns={'ddf': index}, inplace=True)
            fut_ddfs = pd.concat([fut_ddfs, fut_ddfs_region], axis=1)
            
            crrs_cdf_series_fut.rename(columns={'crr': index}, inplace=True)
            fut_cdfs = pd.concat([fut_cdfs, crrs_cdf_series_fut], axis=1)
            crrs_pdf_series_fut.rename(columns={'crr': index}, inplace=True)
            fut_pdfs = pd.concat([fut_pdfs, crrs_pdf_series_fut], axis=1)
            
            
        # ddfs
        fut_ddfs['CZ'] = region
        fut_ddfs.set_index('CZ', append=True, inplace=True)
        fut_ddfs['SSP'] = column_name
        fut_ddfs.set_index('SSP', append=True, inplace=True)

        fut_ddfs_full = fut_ddfs_full.append(fut_ddfs)
        
        
        # cdfs
        fut_cdfs['CZ'] = region
        fut_cdfs.set_index('CZ', append=True, inplace=True)
        fut_cdfs['SSP'] = column_name
        fut_cdfs.set_index('SSP', append=True, inplace=True)

        fut_cdfs_full = fut_cdfs_full.append(fut_cdfs)
        
        
        # pdfs
        fut_pdfs['CZ'] = region
        fut_pdfs.set_index('CZ', append=True, inplace=True)
        fut_pdfs['SSP'] = column_name
        fut_pdfs.set_index('SSP', append=True, inplace=True)

        fut_pdfs_full = fut_pdfs_full.append(fut_pdfs)

print('done!')

/tmp/ipykernel_22582/2405964139.py:17: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, row in column_data.iteritems():
/tmp/ipykernel_22582/2405964139.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fut_ddfs_full = fut_ddfs_full.append(fut_ddfs)
/tmp/ipykernel_22582/2405964139.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fut_cdfs_full = fut_cdfs_full.append(fut_cdfs)
/tmp/ipykernel_22582/2405964139.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fut_pdfs_full = fut_pdfs_full.append(fut_pdfs)
/tmp/ipykernel_22582/2405964139.py:17: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, r

done!


/tmp/ipykernel_22582/2405964139.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fut_ddfs_full = fut_ddfs_full.append(fut_ddfs)
/tmp/ipykernel_22582/2405964139.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fut_cdfs_full = fut_cdfs_full.append(fut_cdfs)
/tmp/ipykernel_22582/2405964139.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fut_pdfs_full = fut_pdfs_full.append(fut_pdfs)


In [64]:
his_ddfs_region

,ddf
mins,
1.2,5.509012
1.8,8.790863
3.0,12.525744
4.2,16.124094
4.8,19.470707
10.2,32.566658
15.0,41.523441
19.8,48.164856
25.2,53.415867


In [65]:
fut_ddfs_full

,,,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
mins,CZ,SSP,,,,,,,,,,,,,,,,,,,,,
1.2,Pilbara,SSP1-1.9,5.8,5.9,5.9,6.0,6.0,6.0,6.1,6.1,6.2,6.2,...,6.5,6.5,6.5,6.5,6.5,6.5,6.5,6.5,6.5,6.5
1.8,Pilbara,SSP1-1.9,9.3,9.4,9.4,9.5,9.6,9.6,9.7,9.8,9.9,9.9,...,10.4,10.4,10.4,10.4,10.4,10.4,10.4,10.4,10.4,10.4
3.0,Pilbara,SSP1-1.9,13.2,13.3,13.4,13.6,13.7,13.8,13.9,14.0,14.1,14.1,...,14.8,14.8,14.8,14.8,14.8,14.8,14.8,14.8,14.8,14.8
4.2,Pilbara,SSP1-1.9,17.0,17.2,17.3,17.5,17.6,17.7,17.9,18.0,18.1,18.2,...,19.1,19.1,19.1,19.1,19.1,19.1,19.1,19.1,19.1,19.1
4.8,Pilbara,SSP1-1.9,20.5,20.7,20.9,21.1,21.3,21.4,21.6,21.7,21.9,22.0,...,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4320.0,Pilbara,SSP5-8.5,322.6,324.5,327.3,330.1,332.9,335.7,338.4,341.2,345.0,347.7,...,687.9,694.5,700.0,705.6,712.1,717.7,724.2,729.8,736.3,741.9
5760.0,Pilbara,SSP5-8.5,328.6,330.5,333.4,336.2,339.1,342.0,344.8,347.7,351.5,354.3,...,702.6,709.3,715.0,720.7,727.3,733.0,739.7,745.4,752.1,757.8
7200.0,Pilbara,SSP5-8.5,331.4,333.3,336.2,339.1,342.0,344.9,347.7,350.6,354.5,357.4,...,709.9,716.7,722.5,728.2,735.0,740.8,747.5,753.3,760.0,765.8


In [66]:
fut_pdfs_full

,,,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
hours,CZ,SSP,,,,,,,,,,,,,,,,,,,,,
0.0,Pilbara,SSP1-1.9,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
0.1,Pilbara,SSP1-1.9,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,...,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009
0.2,Pilbara,SSP1-1.9,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,...,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010
0.3,Pilbara,SSP1-1.9,0.0009,0.0010,0.0010,0.0010,0.0009,0.0009,0.0009,0.0009,0.0009,0.0010,...,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010
0.4,Pilbara,SSP1-1.9,0.0010,0.0010,0.0010,0.0010,0.0011,0.0011,0.0011,0.0011,0.0011,0.0010,...,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23.6,Pilbara,SSP5-8.5,0.0010,0.0011,0.0011,0.0011,0.0011,0.0011,0.0010,0.0010,0.0010,0.0010,...,0.0011,0.0011,0.0011,0.0011,0.0011,0.0011,0.0011,0.0011,0.0011,0.0011
23.7,Pilbara,SSP5-8.5,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0011,0.0011,0.0010,0.0010,...,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010,0.0010
23.8,Pilbara,SSP5-8.5,0.0010,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0010,0.0010,...,0.0010,0.0011,0.0011,0.0010,0.0011,0.0010,0.0011,0.0011,0.0010,0.0010


In [67]:
fut_ddfs_full_file = r'/home/lusun/LuSUN/output/future_regional_ddfs_ARI' + str(ARI) + '_' + str(ens_ptl) + 'th_byNZClimate.csv'
fut_ddfs_full.to_csv(fut_ddfs_full_file, index=True, header=True, float_format='%.8f')

fut_cdfs_full_file = r'/home/lusun/LuSUN/output/future_regional_cdfs_ARI' + str(ARI) + '_' + str(ens_ptl) + 'th_byNZClimate.csv'
fut_cdfs_full.to_csv(fut_cdfs_full_file, index=True, header=True, float_format='%.8f')

fut_pdfs_full_file = r'/home/lusun/LuSUN/output/future_regional_pdfs_ARI' + str(ARI) + '_' + str(ens_ptl) + 'th_byNZClimate.csv'
fut_pdfs_full.to_csv(fut_pdfs_full_file, index=True, header=True, float_format='%.8f')


In [29]:
pdfs_df_his, pdfs_df_fut

NameError: name 'pdfs_df_his' is not defined

In [ ]:
ax = pd.concat([pdfs_df_his*his_ddfs.loc[1440].values, 
                pdfs_df_fut*fut_ddfs.loc[1440].values
                ], axis=1).plot()

In [ ]:
his_ddfs, fut_ddfs

In [ ]:
# get temporal curve from DDFs
df_crrs_his = temporal_curve_24hr(his_ddfs)
df_crrs_fut = temporal_curve_24hr(fut_ddfs)
df_crrs_his, df_crrs_fut

In [ ]:
his_ddfs

In [ ]:
his_ddfs.loc[1440]

In [ ]:
ax = pd.concat([df_crrs_his*his_ddfs.loc[1440].values, 
                df_crrs_fut*fut_ddfs.loc[1440].values
               ], axis=1).plot()
